In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [73]:
#importing input file
df=pd.read_excel('/content/drive/MyDrive/Blackcoffer/Input.xlsx')[['URL_ID','URL']]

In [74]:
df=df.iloc[0:150]
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [ ]:
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  #write title and text to the file
  file_name = '/content/drive/MyDrive/Blackcoffer/TitleText/' + str(url_id) + '.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article)

In [76]:
text_dir = "/content/drive/MyDrive/Blackcoffer/TitleText"
stopwords_dir = "/content/drive/MyDrive/Blackcoffer/StopWords"
sentiment_dir = "/content/drive/MyDrive/Blackcoffer/MasterDictionary"

In [ ]:
import os
stopwords_set = set()

# Iterate over the files in the directory
for filename in os.listdir(stopwords_dir):
    file_path = os.path.join(stopwords_dir, filename)
    try:
       with open(os.path.join(stopwords_dir,filename),'r',encoding='ISO-8859-1') as file:
            # Read the file and add its contents to the set of stop words
            stopwords_set.update(file.read().split())
    except IOError as e:
        print(f"Error reading file: {file_path}\n{str(e)}")

# Print the set of stop words
print(stopwords_set)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

In [79]:
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r') as f:
    text = f.read()
    words = word_tokenize(text)
    filtered_text = [word for word in words if word.lower() not in stopwords_set]
    docs.append(filtered_text)

In [ ]:
docs

In [81]:
positive_words = set()
negative_words = set()

# Iterate over the files in the directory
for filename in os.listdir(sentiment_dir):
    file_path = os.path.join(sentiment_dir, filename)
    try:
        with open(file_path, 'r',encoding='ISO-8859-1') as file:
            # Read the file and add its contents to the appropriate word set
            word_list = file.read().split()
            if 'positive' in filename:
                positive_words.update(word_list)
            elif 'negative' in filename:
                negative_words.update(word_list)
    except IOError as e:
        print(f"Error reading file: {file_path}\n{str(e)}")

In [ ]:
print("Positive words:")
for word in positive_words:
    print(word)

In [ ]:
tokens = text.lower().split()

# Calculate the positive score
positive_score = sum(1 for token in tokens if token in positive_words)

# Print the positive score
print("Positive Score:", positive_score)

In [ ]:
tokens = text.lower().split()

# Calculate the negative score
negative_score = sum(1 for token in tokens if token in negative_words)

# Print the negative score
print("Negative Score:", negative_score)

In [ ]:
#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Polarity_Score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)
print('polarity_score=', Polarity_Score)
     

In [86]:
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in positive_words])
  Negative_words.append([word for word in docs[i] if word.lower() in negative_words])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

In [ ]:
import nltk.data

avg_sentence_length = []



# Tokenize the text into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(text)

# Calculate the average sentence length
total_sentence_length = sum(len(sentence.split()) for sentence in sentences)
average_sentence_length = total_sentence_length / len(sentences)

# Print the average sentence length
print("Average Sentence Length:", average_sentence_length)


In [ ]:
complex_words = []

# Define the complexity criterion
complex_word_length = 3 

# Tokenize the text
tokens = text.lower().split()

# Calculate the count of complex words
complex_word_count = sum(1 for token in tokens if len(token) > complex_word_length)
percentage_complex_words = (complex_word_count / len(words)) * 100
# Print the count of complex words
print("Complex Word Count:", complex_word_count)
print("Percentage of Complex Words:", percentage_complex_words)


In [ ]:
#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
Fog_Index = 0.4 * (average_sentence_length + percentage_complex_words)
print('fog index= ',Fog_Index )

In [ ]:

tokens = text.split()

# Calculate the word count
word_count = len(tokens)

# Print the word count
print("Word Count:", word_count)


In [ ]:
import nltk

# Tokenize the text into sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(text)

# Calculate the total number of words and sentences
total_words = 0
total_sentences = len(sentences)

# Iterate over sentences and count words
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    total_words += len(words)

# Calculate the average number of words per sentence
average_words_per_sentence = total_words / total_sentences

# Print the average number of words per sentence
print("Average Words Per Sentence:", average_words_per_sentence)


In [ ]:
pip install pyphen

In [ ]:
import pyphen

# Create an instance of the Pyphen hyphenation object
dic = pyphen.Pyphen(lang='en')

# Tokenize the text
tokens = text.lower().split()

# Calculate the syllable count per word
syllable_counts = []
for token in tokens:
    syllables = dic.inserted(token).count('-') + 1
    syllable_counts.append(syllables)

# Print the syllable count per word
for i, token in enumerate(tokens):
    print(f"Word: {token} | Syllable Count: {syllable_counts[i]}")
# Count the number of syllables in the word
syllable_count = len(dic.inserted(word).split('-'))

# Print the syllable count
print("Syllable Count:", syllable_count)

In [ ]:


# List of personal pronouns
personal_pronouns = ['I', 'me', 'my', 'mine', 'myself',
                     'you', 'your', 'yours', 'yourself',
                     'he', 'him', 'his', 'himself',
                     'she', 'her', 'hers', 'herself',
                     'it', 'its', 'itself',
                     'we', 'us', 'our', 'ours', 'ourselves',
                     'they', 'them', 'their', 'theirs', 'themselves']

# Count occurrences of personal pronouns
pronoun_count = 0
for pronoun in personal_pronouns:
    pronoun_count += text.lower().count(pronoun)

# Print the count of personal pronouns
print("Count of Personal Pronouns:", pronoun_count)


In [ ]:
# Tokenize the text
words = text.split()

# Calculate the total word length
total_word_length = sum(len(word) for word in words)

# Calculate the average word length
average_word_length = total_word_length / len(words)

# Print the average word length
print("Average Word Length:", average_word_length)


In [101]:
data={'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':Polarity_Score,'subjectiivity_score':subjectivity_score,'avg_senetence_length':average_sentence_length,'Percentage_of_Complex_words':percentage_complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':average_words_per_sentence,'complex_Word_Count':complex_word_count,'word_count':word_count,'syllable_count':syllable_count,'personal_pronouns':pronoun_count,'avg_word_length':average_word_length}

In [ ]:
output=pd.DataFrame()
output=output.append(data,ignore_index=True)
#output.columns=['positive_score','negative_score','Polarity_Score','subjectiivity_score','avg_senetence_length','Percentage_of_Complex_words','Fog_Index','avg_no_of_words_per_sentence','complex_Word_Count','word_count','syllable_count','personal_pronouns','avg_word_length']
output

In [ ]:
with open('output.csv', 'a') as f:#creating text file 
   output.to_csv(f, index=False, header=False)
files.download('output.csv')